In [1]:
import pandas as pd
import numpy as np
from IPython.display import display as dis
from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)
import pandas.plotting._matplotlib

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 30)
pd.options.display.width = 170

In [2]:
movies = pd.read_csv('movies_metadata.csv')
movies.head()
movies.shape

(45466, 24)

In [3]:
movies = movies.loc[((movies.budget != str(0)) & (movies.budget != int(0))) & ((movies.revenue != str(0)) & (movies.revenue != int(0)))]
movies.shape

(5384, 24)

In [4]:
to_replace = ['{','}','[',']',"'",':','1','2','3','4','5','6','7','8','9','0','name','id',' ']

for element in to_replace:
    movies.genres = movies.genres.str.replace(element,'')
    movies.genres = movies.genres.str.replace(',,',',')
    movies.production_companies = movies.production_companies.str.replace(element, '')
    movies.production_companies = movies.production_companies.str.replace(',,',',')

movies.genres = movies.genres.str.strip(', \n\t')
movies.production_companies = movies.production_companies.str.strip(', \n\t')
print(movies.shape)
movies = movies[(movies.genres != '') & (movies.production_companies != '')]
print(movies.shape)

movies.genres = movies.genres.apply(lambda x: ','.join(sorted(x.split(','))))
movies.production_companies = movies.production_companies.apply(lambda x: ','.join(sorted(x.split(','))))

print(movies.genres.head(10))
print(movies.production_companies.head(10))

(5384, 24)
(5202, 24)
0         Animation,Comedy,Family
1        Adventure,Family,Fantasy
3            Comedy,Drama,Romance
5     Action,Crime,Drama,Thriller
8       Action,Adventure,Thriller
9       Action,Adventure,Thriller
10           Comedy,Drama,Romance
13                  Drama,History
14               Action,Adventure
15                    Crime,Drama
Name: genres, dtype: object
0                                 PixarAnimationStudios
1     InterscopeCommunications,TeitlerFilm,TriStarPi...
3                    TwentiethCenturyFoxFilmCorporation
5            ForwardPass,RegencyEnterprises,WarnerBros.
8     ImperialEntertainment,SignatureEntertainment,U...
9                          EonProductions,UnitedArtists
10             CastleRockEntertainment,ColumbiaPictures
13       CinergiPicturesEntertainment,HollywoodPictures
14    CarolcoPictures,LaurenceMarkProductions,LeStud...
15    DeFina-Cappa,LégendeEntreprises,SyalisDA,Unive...
Name: production_companies, dtype: object


In [5]:
dis(movies.shape)
genres_exploded = movies
genres_exploded.genres = genres_exploded.genres.apply(lambda x: x.split(','))
genres_exploded = genres_exploded.explode('genres')
genres_exploded.shape

(5202, 24)

(13639, 24)

In [6]:
dis(movies.shape)
prod_exploded = movies
prod_exploded.production_companies = prod_exploded.production_companies.apply(lambda x: x.split(','))
prod_exploded = prod_exploded.explode('production_companies')
prod_exploded.shape

(5202, 24)

(15892, 24)

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # dis(genres_exploded.genres.value_counts())
    # dis(movies.genres.value_counts())
    dis(prod_exploded.production_companies.value_counts())

WarnerBros.                                                                         418
UniversalPictures                                                                   386
ParamountPictures                                                                   340
TwentiethCenturyFoxFilmCorporation                                                  296
ColumbiaPictures                                                                    210
NewLineCinema                                                                       174
Metro-Goldwyn-Mayer(MGM)                                                            164
WaltDisneyPictures                                                                  132
TouchstonePictures                                                                  129
ColumbiaPicturesCorporation                                                         117
RelativityMedia                                                                     113
MiramaxFilms                    

In [8]:
genres_exploded = genres_exploded.groupby('genres').filter(lambda x: x.genres.size > 1)

In [9]:
movies.to_csv('movies_metadata_modified.csv')
genres_exploded.to_csv('genres_exploded.csv')
prod_exploded.to_csv('prod_exploded.csv')

In [10]:
from scipy import stats
from collections import OrderedDict

dict1 = OrderedDict()
for value in genres_exploded.genres.unique():
    onedf = genres_exploded.groupby('genres')['revenue'].get_group(value).astype('int')
    twodf = genres_exploded.groupby('genres')['budget'].get_group(value).astype('int')
    score = (onedf.sub(twodf)).div(twodf)
    dict1[value] = stats.percentileofscore(score,0)

dict2 = sorted((100 - v,k) for k,v in dict1.items())
dict2
    

[(20.038535645472063, 'Family'),
 (21.724137931034484, 'Animation'),
 (23.442449841605068, 'Adventure'),
 (24.554455445544555, 'Fantasy'),
 (26.475694444444443, 'Horror'),
 (27.476038338658146, 'ScienceFiction'),
 (27.64341957255343, 'Comedy'),
 (27.689530685920577, 'Action'),
 (28.089887640449437, 'Western'),
 (28.571428571428573, 'War'),
 (28.88888888888889, 'Documentary'),
 (29.290853031860227, 'Romance'),
 (29.422169811320753, 'Crime'),
 (30.27027027027027, 'Music'),
 (31.220338983050848, 'Thriller'),
 (33.624454148471614, 'History'),
 (34.240362811791385, 'Mystery'),
 (34.411410204901564, 'Drama'),
 (47.82608695652174, 'Foreign')]

In [11]:
pd.Series(dict1).to_csv('profitable_genres.csv')

In [29]:
dict3 = {}
for value in prod_exploded.groupby('production_companies').filter(lambda x: x.production_companies.size >= 20).production_companies.unique():
    onedf = prod_exploded.groupby('production_companies')['revenue'].get_group(value).astype('int')
    twodf = prod_exploded.groupby('production_companies')['budget'].get_group(value).astype('int')
    score = (onedf.sub(twodf)).div(twodf)
    dict3[value] = stats.percentileofscore(score,0)

dict4 = sorted((100 - v,k) for k,v in dict3.items())
dict4

[(24.24242424242425, 'MorganCreekProductions'),
 (35.0, 'IntermediaFilms'),
 (38.70967741935484, 'MillenniumFilms'),
 (43.58974358974359, 'FranceCinéma'),
 (44.44444444444444, 'EpsilonMotionPictures'),
 (46.15384615384615, 'HollywoodPictures'),
 (50.0, 'FilmNationEntertainment'),
 (53.84615384615385, 'Canal+'),
 (57.142857142857146, 'PolyGramFilmedEntertainment'),
 (58.62068965517241, 'UKFilmCouncil'),
 (59.375, 'LakeshoreEntertainment'),
 (61.111111111111114, 'OrionPictures'),
 (61.19402985074627, 'StudioCanal'),
 (64.0, 'TFFilmsProduction'),
 (64.04494382022472, 'TriStarPictures'),
 (64.86486486486487, 'BBCFilms'),
 (68.18181818181819, 'ImagineEntertainment'),
 (68.18181818181819, 'NPVEntertainment'),
 (69.23076923076923, 'Film'),
 (69.44444444444444, 'ScottFreeProductions'),
 (69.44444444444444, 'SpyglassEntertainment'),
 (69.56521739130434, 'Pictures'),
 (69.76744186046511, 'TouchstonePictures'),
 (70.0, 'AmericanZoetrope'),
 (70.37037037037037, 'ParticipantMedia'),
 (71.4285714285

In [30]:
pd.Series(dict3).to_csv('profitable_companies.csv')